In [1]:
## Importing Libraries
from sklearn.neural_network import MLPClassifier
import pandas as pd
from sklearn.model_selection import train_test_split
import numpy as np
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix
import pickle
from sklearn.tree import DecisionTreeClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
import joblib
import pickle
from sklearn.utils import shuffle
from sklearn.ensemble import RandomForestClassifier
from xgboost import XGBClassifier
from tensorflow.keras import Sequential,layers, losses, optimizers
from tensorflow.keras.layers import Dense
import tensorflow as tf
import time

In [2]:
## Load CICIDS 2017 training dataset
df_train = pd.read_csv("all_group_train_normalized.csv")

In [3]:
df_train['Label'].value_counts()

Benign          53432
DoS              8906
Web Attack       8906
Infiltration     8905
Port Scan        8905
DDoS             8905
Brute Force      8905
Name: Label, dtype: int64

In [4]:
## Loading CICIDS217 test dataset
df_test = pd.read_csv("all_group_test_normalized.csv")

In [5]:
## Subsetting attack packets from CICIDS 2017 dataset
infiltration_attack = df_test[df_test['Label'] == 'Infiltration'].drop(['Label_binary','Label'], axis=1).iloc[:,0:200]
print(infiltration_attack.shape)
portscan_attack = df_test[df_test['Label'] == 'Port Scan'].drop(['Label_binary','Label'], axis=1).iloc[:,0:200]
print(portscan_attack.shape)
ddos_attack = df_test[df_test['Label'] == 'DDoS'].drop(['Label_binary','Label'], axis=1).iloc[:,0:200]
print(ddos_attack.shape)
bruteforce_attack = df_test[df_test['Label'] == 'Brute Force'].drop(['Label_binary','Label'], axis=1).iloc[:,0:200]
print(bruteforce_attack.shape)
dos_attack = df_test[df_test['Label'] == 'DoS'].drop(['Label_binary','Label'], axis=1).iloc[:,0:200]
print(dos_attack.shape)
web_attack = df_test[df_test['Label'] == 'Web Attack'].drop(['Label_binary','Label'], axis=1).iloc[:,0:200]
print(web_attack.shape)
benign_data = df_test[df_test['Label'] == 'Benign'].drop(['Label_binary','Label'], axis=1).iloc[:,0:200]
print(benign_data.shape)

(3817, 200)
(3817, 200)
(3817, 200)
(3817, 200)
(3816, 200)
(3816, 200)
(22900, 200)


In [6]:
infiltration_attack

,ip_header_byte_2,ip_header_byte_3,ip_header_byte_4,ip_header_byte_5,ip_header_byte_6,ip_header_byte_7,ip_header_byte_8,ip_header_byte_10,ip_header_byte_11,tcp_header_byte_4,...,tcp_segment_data_byte_125,tcp_segment_data_byte_126,tcp_segment_data_byte_127,tcp_segment_data_byte_128,tcp_segment_data_byte_129,tcp_segment_data_byte_130,tcp_segment_data_byte_131,tcp_segment_data_byte_132,tcp_segment_data_byte_133,tcp_segment_data_byte_134
5,0.007843,0.109804,0.054902,0.980392,0.25098,0.0,0.501961,0.674510,0.223529,0.196078,...,0.152941,0.431373,0.427451,0.380392,0.439216,0.152941,0.125490,0.411765,0.450980,0.125490
14,0.003922,0.717647,0.286275,0.760784,0.25098,0.0,0.501961,0.443137,0.839216,0.929412,...,0.152941,0.431373,0.427451,0.380392,0.439216,0.152941,0.125490,0.411765,0.450980,0.125490
16,0.003922,0.835294,0.172549,0.192157,0.25098,0.0,0.501961,0.560784,0.286275,0.196078,...,0.050980,0.039216,0.262745,0.227451,0.360784,0.243137,0.431373,0.427451,0.380392,0.439216
19,0.007843,0.109804,0.054902,0.494118,0.25098,0.0,0.501961,0.674510,0.709804,0.196078,...,0.152941,0.431373,0.427451,0.380392,0.439216,0.152941,0.125490,0.411765,0.450980,0.125490
28,0.007843,0.109804,0.145098,0.443137,0.25098,0.0,0.501961,0.584314,0.760784,0.929412,...,0.152941,0.431373,0.427451,0.380392,0.439216,0.152941,0.125490,0.411765,0.450980,0.125490
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
45759,0.003922,0.133333,0.215686,0.541176,0.25098,0.0,0.501961,0.517647,0.639216,0.196078,...,0.050980,0.039216,0.262745,0.227451,0.360784,0.243137,0.431373,0.427451,0.380392,0.439216
45761,0.007843,0.686275,0.180392,0.145098,0.25098,0.0,0.501961,0.549020,0.482353,0.196078,...,0.152941,0.431373,0.427451,0.380392,0.439216,0.152941,0.125490,0.411765,0.450980,0.125490
45768,0.000000,0.270588,0.086275,0.329412,0.25098,0.0,0.501961,0.650980,0.713725,0.196078,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
45781,0.011765,0.168627,0.462745,0.356863,0.25098,0.0,0.501961,0.262745,0.788235,0.192157,...,0.450980,0.298039,0.125490,0.176471,0.431373,0.125490,0.192157,0.223529,0.196078,0.180392


In [7]:
## Loading CICIDS 2018 training data
df_test_2018 = pd.read_csv("all_group_train_normalized_2018.csv")

In [8]:
df_test_2018 = df_test_2018.sample(frac = 1).reset_index(drop = True)

In [9]:
df_test_2018

,ip_header_byte_2,ip_header_byte_3,ip_header_byte_4,ip_header_byte_5,ip_header_byte_6,ip_header_byte_7,ip_header_byte_8,ip_header_byte_10,ip_header_byte_11,tcp_header_byte_4,...,tcp_segment_data_byte_1452,tcp_segment_data_byte_1453,tcp_segment_data_byte_1454,tcp_segment_data_byte_1455,tcp_segment_data_byte_1456,tcp_segment_data_byte_1457,tcp_segment_data_byte_1458,tcp_segment_data_byte_1459,Label,Label_binary
0,0.003922,0.576471,0.219608,0.678431,0.25098,0.0,0.250980,0.537255,0.400000,0.419608,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,Web Attack,Malicious
1,0.007843,0.792157,0.839216,0.407843,0.25098,0.0,0.250980,0.541176,0.152941,0.333333,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,DoS,Malicious
2,0.000000,0.156863,0.309804,0.772549,0.25098,0.0,0.501961,0.807843,0.713725,0.356863,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,Benign,Benign
3,0.000000,0.156863,0.168627,0.537255,0.25098,0.0,0.250980,0.172549,0.870588,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,Infiltration,Malicious
4,0.000000,0.156863,0.188235,0.603922,0.25098,0.0,0.501961,0.231373,0.847059,0.274510,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,Benign,Benign
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
291153,0.000000,0.235294,0.023529,0.450980,0.25098,0.0,0.247059,0.337255,0.184314,0.803922,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,Brute Force,Malicious
291154,0.000000,0.156863,0.372549,0.388235,0.25098,0.0,0.501961,0.749020,0.094118,0.356863,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,Benign,Benign
291155,0.000000,0.976471,0.058824,0.298039,0.25098,0.0,0.415686,0.482353,0.219608,0.937255,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,Infiltration,Malicious
291156,0.000000,0.156863,0.372549,0.498039,0.25098,0.0,0.501961,0.745098,0.988235,0.356863,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,Benign,Benign


In [10]:
X_train_2018 = df_test_2018.iloc[:,0:1525]
Y_train_2018 = df_test_2018.iloc[:,1526:1527]

In [11]:
X_train_2017 = df_test.iloc[:,0:1525]
Y_train_2017 = df_test.iloc[:,1526:1527]

In [12]:
Y_train_2017.loc[Y_train_2017['Label_binary'] == 'Benign', 'Label_binary'] = 0
Y_train_2017.loc[Y_train_2017['Label_binary'] == 'Malicious', 'Label_binary'] = 1

In [13]:
Y_train_2018.loc[Y_train_2018['Label_binary'] == 'Benign', 'Label_binary'] = 0
Y_train_2018.loc[Y_train_2018['Label_binary'] == 'Malicious', 'Label_binary'] = 1

In [14]:
X_train_total = pd.concat([X_train_2017,X_train_2018])
Y_train_total = pd.concat([Y_train_2017,Y_train_2018])

In [15]:
X_train_ = X_train_total.values.astype('float32')
Y_train_ = Y_train_total.values.astype('float32')

In [16]:
## Subsetting atatck apckets from CICIDS 2018 dataset
infiltration_attack_2018 = df_test_2018[df_test_2018['Label'] == 'Infiltration'].drop(['Label_binary','Label'], axis=1).iloc[:,0:200]
print(infiltration_attack_2018.shape)
portscan_attack_2018 = df_test_2018[df_test_2018['Label'] == 'Port Scan'].drop(['Label_binary','Label'], axis=1).iloc[:,0:200]
print(portscan_attack_2018.shape)
ddos_attack_2018 = df_test_2018[df_test_2018['Label'] == 'ddos'].drop(['Label_binary','Label'], axis=1).iloc[:,0:200]
print(ddos_attack_2018.shape)
bruteforce_attack_2018 = df_test_2018[df_test_2018['Label'] == 'Brute Force'].drop(['Label_binary','Label'], axis=1).iloc[:,0:200]
print(bruteforce_attack_2018.shape)
dos_attack_2018 = df_test_2018[df_test_2018['Label'] == 'DoS'].drop(['Label_binary','Label'], axis=1).iloc[:,0:200]
print(dos_attack_2018.shape)
web_attack_2018 = df_test_2018[df_test_2018['Label'] == 'Web Attack'].drop(['Label_binary','Label'], axis=1).iloc[:,0:200]
print(web_attack_2018.shape)
benign_data_2018 = df_test_2018[df_test_2018['Label'] == 'Benign'].drop(['Label_binary','Label'], axis=1).iloc[:,0:200]
print(benign_data_2018.shape)

(29116, 200)
(0, 200)
(29115, 200)
(29116, 200)
(29116, 200)
(29116, 200)
(145579, 200)


In [17]:
benign_data_total = pd.concat([benign_data, benign_data_2018])

In [18]:
# Y_train.loc[Y_train['Label_binary'] == 'Benign', 'Label_binary'] = 0

In [19]:
## Loading the baseline stage 2 AE model
stage2_AE = tf.keras.models.load_model('Stage2_AE.h5')

In [20]:
## the pbRe reconstruction metric
def recon_metrices(data, reconstructed_data):

    maes = np.absolute(data.values - reconstructed_data)
    
    for j in range(len(maes)):
        m = maes[j]
        d = data.values[j]
        r = reconstructed_data[j]
        for el in range(len(m)):
            if r[el] == 0 or d[el] == 0:
                m[el] = 0
 
            
    mae_byte_list = []
    ip_mae_list = []
    tcp_header_mae_list = []
    tcp_options_mae_list = []
    tcp_segment_mae_list = []
    i=0
    for mae in maes:
#         print(np.count_nonzero(reconstructed_data[i]))
#         print(np.count_nonzero(data.values[i]))
#         print(np.max(np.count_nonzero(reconstructed_data[i]),np.count_nonzero(data.values[i])))
        mae_byte = sum(mae) / np.count_nonzero(mae)
        mae_byte_list.append(mae_byte)
        i += 1
    return mae_byte_list

In [23]:
inf = stage2_AE.predict(infiltration_attack.values.astype('float32'))

120/120 [==============================] - 1s 3ms/step


In [24]:
mae_byte_list = recon_metrices(infiltration_attack,inf)

In [25]:
## Classification
count_99 = len([i for i in mae_byte_list if i > 0.09])
print(count_99 / len(mae_byte_list))

0.9992140424417082


In [26]:
### Retraining with CICIDS 2018 data

In [90]:
## Modifiable layers
mod_layers = [6,7,8,10,11,12]

In [91]:
## Modifiable neurons in modifiable layers
layer6_ind = [h for h in range(0,50,3)]
layer7_ind = [h for h in range(0,25,3)]
layer8_ind = [h for h in range(0,12,3)]
layer9_ind = []
layer10_ind = [h for h in range(0,11,3)]
layer11_ind = [h for h in range(0,25,3)]
layer12_ind = [h for h in range(0,50,3)]

In [92]:
## Initializing a new autoencoder model
retrained_stage2 = tf.keras.Sequential()

In [93]:
## Architecture of the retrained model
for i, layer in enumerate(stage2_AE.layers):
    if i not in mod_layers:
        layer.trainable = False
        retrained_stage2.add(layer)
    else:
        retrained_stage2.add(layer)
        if i == 6:
            for ind in range(0,50):
                if ind in layer6_ind:
                    retrained_stage2.layers[-1].kernel[ind]._trainable = False
            retrained_stage2.add(Dense(50, activation='relu', name = 'Trainable1'))
        if i == 7:
            for ind in range(0,25):
                if ind in layer7_ind:
                    retrained_stage2.layers[-1].kernel[ind]._trainable = False
                    
        if i == 8:
            for ind in range(0,12):
                if ind in layer8_ind:
                    retrained_stage2.layers[-1].kernel[ind]._trainable = False
        
        if i == 10:
            for ind in range(0,6):
                if ind in layer10_ind:
                    retrained_stage2.layers[-1].kernel[ind]._trainable = False
        
        if i == 11:
            for ind in range(0,12):
                if ind in layer11_ind:
                    retrained_stage2.layers[-1].kernel[ind]._trainable = False
        if i == 12:
            for ind in range(0,25):
                if ind in layer12_ind:
                    retrained_stage2.layers[-1].kernel[ind]._trainable = False
            retrained_stage2.add(Dense(50, activation='relu', name = 'Trainable2'))


In [94]:
retrained_stage2.summary()

Model: "sequential_12"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 200)               40200     
                                                                 
 dense_1 (Dense)             (None, 150)               30150     
                                                                 
 leaky_re_lu (LeakyReLU)     (None, 150)               0         
                                                                 
 dense_2 (Dense)             (None, 150)               22650     
                                                                 
 leaky_re_lu_1 (LeakyReLU)   (None, 150)               0         
                                                                 
 dense_3 (Dense)             (None, 100)               15100     
                                                                 
 dense_4 (Dense)             (None, 50)              

In [95]:
## Compiling and Training
start_time = time.time()
retrained_stage2.compile(optimizer='adam', loss='mae')
retrained_stage2.fit(benign_data_total , benign_data_total , batch_size=256, epochs=500, shuffle=True)
end_time = time.time()

Epoch 1/500
659/659 [==============================] - 4s 5ms/step - loss: 0.0434
Epoch 2/500
659/659 [==============================] - 3s 5ms/step - loss: 0.0418
Epoch 3/500
659/659 [==============================] - 3s 5ms/step - loss: 0.0411
Epoch 4/500
659/659 [==============================] - 3s 5ms/step - loss: 0.0408
Epoch 5/500
659/659 [==============================] - 3s 5ms/step - loss: 0.0405
Epoch 6/500
659/659 [==============================] - 3s 5ms/step - loss: 0.0404
Epoch 7/500
659/659 [==============================] - 3s 5ms/step - loss: 0.0402
Epoch 8/500
659/659 [==============================] - 3s 5ms/step - loss: 0.0401
Epoch 9/500
659/659 [==============================] - 3s 5ms/step - loss: 0.0400
Epoch 10/500
659/659 [==============================] - 3s 5ms/step - loss: 0.0399
Epoch 11/500
659/659 [==============================] - 3s 5ms/step - loss: 0.0398
Epoch 12/500
659/659 [==============================] - 3s 5ms/step - loss: 0.0397
Epoch 13/500


659/659 [==============================] - 3s 5ms/step - loss: 0.0296
Epoch 197/500
659/659 [==============================] - 3s 5ms/step - loss: 0.0296
Epoch 198/500
659/659 [==============================] - 3s 5ms/step - loss: 0.0296
Epoch 199/500
659/659 [==============================] - 3s 5ms/step - loss: 0.0296
Epoch 200/500
659/659 [==============================] - 3s 5ms/step - loss: 0.0296
Epoch 201/500
659/659 [==============================] - 3s 5ms/step - loss: 0.0296
Epoch 202/500
659/659 [==============================] - 3s 5ms/step - loss: 0.0296
Epoch 203/500
659/659 [==============================] - 4s 5ms/step - loss: 0.0296
Epoch 204/500
659/659 [==============================] - 3s 5ms/step - loss: 0.0296
Epoch 205/500
659/659 [==============================] - 3s 5ms/step - loss: 0.0296
Epoch 206/500
659/659 [==============================] - 3s 5ms/step - loss: 0.0296
Epoch 207/500
659/659 [==============================] - 3s 5ms/step - loss: 0.0295
Epoch 

659/659 [==============================] - 6s 9ms/step - loss: 0.0293
Epoch 391/500
659/659 [==============================] - 7s 11ms/step - loss: 0.0293
Epoch 392/500
659/659 [==============================] - 6s 9ms/step - loss: 0.0293
Epoch 393/500
659/659 [==============================] - 7s 10ms/step - loss: 0.0293
Epoch 394/500
659/659 [==============================] - 6s 9ms/step - loss: 0.0293
Epoch 395/500
659/659 [==============================] - 6s 8ms/step - loss: 0.0293
Epoch 396/500
659/659 [==============================] - 7s 11ms/step - loss: 0.0293
Epoch 397/500
659/659 [==============================] - 5s 8ms/step - loss: 0.0293
Epoch 398/500
659/659 [==============================] - 6s 9ms/step - loss: 0.0293
Epoch 399/500
659/659 [==============================] - 6s 9ms/step - loss: 0.0293
Epoch 400/500
659/659 [==============================] - 5s 7ms/step - loss: 0.0293
Epoch 401/500
659/659 [==============================] - 6s 9ms/step - loss: 0.0293
Epo

In [96]:
## Saving the retrained model
retrained_stage2.save('retrained_stage2_hybrid.h5')

In [21]:
## Loading the retrained model
from tensorflow.keras.models import load_model

retrained_stage2 = load_model('retrained_stage2_hybrid.h5')

In [97]:
time_taken = end_time - start_time
print(time_taken)

2039.2511141300201


In [ ]:
## CICIDS2017 and CICIDS2018 Testing retrained model

In [22]:
inf_ = retrained_stage2.predict(infiltration_attack)
port_ = retrained_stage2.predict(portscan_attack)
dos_ = retrained_stage2.predict(dos_attack)
ddos_ = retrained_stage2.predict(ddos_attack)
web_ = retrained_stage2.predict(web_attack)
brute_ = retrained_stage2.predict(bruteforce_attack)
ben_ = retrained_stage2.predict(benign_data)

716/716 [==============================] - 2s 2ms/step


In [23]:
mae_byte_list = recon_metrices(benign_data,ben_)

In [24]:
mae_byte_avg = sum(mae_byte_list) / len(mae_byte_list)
print('mae_byte_avg', mae_byte_avg)
mae_byte_per_95 = np.percentile(mae_byte_list, 99)
print('mae_byte_99_percentile', mae_byte_per_95)
mae_byte_per_95 = np.percentile(mae_byte_list, 95)
print('mae_byte_95_percentile', mae_byte_per_95)
mae_byte_per_95 = np.percentile(mae_byte_list, 90)
print('mae_byte_90_percentile', mae_byte_per_95)

mae_byte_avg 0.11650496069633351
mae_byte_99_percentile 0.22918970822951318
mae_byte_95_percentile 0.17450515541714123
mae_byte_90_percentile 0.15900632962172317


In [25]:
inf_2018 = retrained_stage2.predict(infiltration_attack_2018)
dos_2018 = retrained_stage2.predict(dos_attack_2018)
ddos_2018 = retrained_stage2.predict(ddos_attack_2018)
web_2018 = retrained_stage2.predict(web_attack_2018)
brute_2018 = retrained_stage2.predict(bruteforce_attack_2018)
ben_2018 = retrained_stage2.predict(benign_data_2018)

4550/4550 [==============================] - 12s 3ms/step


In [26]:
mae_byte_list = recon_metrices(benign_data_2018,ben_2018)

In [28]:
## Classification
count_99 = len([i for i in mae_byte_list if i > 0.22])
print(count_99 / len(mae_byte_list))

0.10956250558116212


In [156]:
print('Infiltration')
cnt_inf, acc_inf = accuracy_calc(inf_2018)
print(cnt_inf)
print(acc_inf)
print('DoS')
cnt_dos, acc_dos = accuracy_calc(dos_2018)
print(cnt_dos)
print(acc_dos)
print('DDoS')
cnt_ddos, acc_ddos = accuracy_calc(ddos_2018)
print(cnt_ddos)
print(acc_ddos)
print('Web')
cnt_web, acc_web = accuracy_calc(web_2018)
print(cnt_web)
print(acc_web)
print('Brute')
cnt_brute, acc_brute = accuracy_calc(brute_2018)
print(cnt_brute)
print(acc_brute)
print('Benign')
cnt_ben, acc_ben = accuracy_calc(ben_2018)
print(cnt_ben)
print(1-acc_ben)

Infiltration
7089
0.7565256216513258
DoS
1578
0.9458029949168842
DDoS
3
0.999896960329727
Web
1476
0.9493062233823327
Brute
730
0.9749278747080643
Benign
130481
0.8962899868799759


In [ ]:
## CICIDS2017 Testing Retrained Model

In [157]:
inf = retrained_stage1.predict(infiltration_attack)
dos = retrained_stage1.predict(dos_attack)
ddos = retrained_stage1.predict(ddos_attack)
web = retrained_stage1.predict(web_attack)
brute = retrained_stage1.predict(bruteforce_attack)
ben = retrained_stage1.predict(benign_data)

716/716 [==============================] - 3s 4ms/step


In [158]:
print('Infiltration')
cnt_inf, acc_inf = accuracy_calc(inf)
print(cnt_inf)
print(acc_inf)
print('DoS')
cnt_dos, acc_dos = accuracy_calc(dos)
print(cnt_dos)
print(acc_dos)
print('DDoS')
cnt_ddos, acc_ddos = accuracy_calc(ddos)
print(cnt_ddos)
print(acc_ddos)
print('Web')
cnt_web, acc_web = accuracy_calc(web)
print(cnt_web)
print(acc_web)
print('Brute')
cnt_brute, acc_brute = accuracy_calc(brute)
print(cnt_brute)
print(acc_brute)
print('Benign')
cnt_ben, acc_ben = accuracy_calc(ben)
print(cnt_ben)
print(1-acc_ben)

Infiltration
3
0.9992140424417082
DoS
170
0.9554507337526206
DDoS
50
0.9869007073618025
Web
99
0.9740566037735849
Brute
52
0.9863767356562746
Benign
22597
0.986768558951965
